# 1. Main Thread:

- The main thread is the thread in which the Python program starts execution. When a Python program runs, it always begins by executing the code in the main thread. If no additional threads are created, the main thread will complete the execution of the program.

### Key Points About Main Thread:
- Starting Point: The main thread is created when the program starts and is where the initial execution of the program occurs.
- Primary Thread: It is the thread from which all other threads are spawned (if created).
- Program Terminates: The main thread controls the termination of the program. If the main thread completes execution, the program will exit, unless there are daemon threads running.

### Main Thread Attributes:
- name: The name of the current thread, which for the main thread is typically "MainThread".
- is_alive(): Returns True if the main thread is still running, otherwise False.
- ident: The unique identifier for the thread (it is set after the thread starts).
- native_id: The native OS thread ID, which can be useful for OS-level debugging.
- threading.active_count(): Returns the number of Thread objects currently alive (i.e., threads that have been started and not yet terminated).
- threading.current_thread(): Returns the current thread object, which refers to the thread calling the method. This is typically the main thread, unless explicitly called from another thread.

In [6]:
import threading
import time

# Main thread accesses its own attributes before any additional threads are started
print(f"thread name: {threading.current_thread().name}")
print(f"thread alive: {threading.current_thread().is_alive()}")
print(f"thread ident: {threading.current_thread().ident}")
print(f"thread native ID: {threading.current_thread().native_id}")


thread name: MainThread
thread alive: True
thread ident: 22396
thread native ID: 22396


In [7]:
import threading
import time

def child_thread():
    print(f"Child thread {threading.current_thread().name} is starting...")
    time.sleep(2)  # Simulate some work
    print(f"Child thread {threading.current_thread().name} is finishing...")

def main_thread():
    print("Main thread is starting...")

    # Check the active thread count before starting the child threads
    print(f"Active threads before starting child threads: {threading.active_count()}")

    # Create and start child threads
    threads = []
    for i in range(3):
        thread = threading.Thread(target=child_thread, name=f"Child-{i+1}")
        threads.append(thread)
        thread.start()

    # Check the active thread count after starting child threads
    print(f"Active threads after starting child threads: {threading.active_count()}")

    # Wait for all child threads to finish
    for thread in threads:
        thread.join()


    # Check the active thread count again after all threads finish
    print(f"Active threads after finishing all threads: {threading.active_count()}")

    print("Main thread is finishing...")


# Run the main thread
main_thread()


Main thread is starting...
Active threads before starting child threads: 6
Child thread Child-1 is starting...
Child thread Child-2 is starting...
Child thread Child-3 is starting...
Active threads after starting child threads: 9
Child thread Child-1 is finishing...Child thread Child-2 is finishing...
Child thread Child-3 is finishing...

Active threads after finishing all threads: 6
Main thread is finishing...


In [8]:
print(threading.active_count())

6


# 2. Daemon Thread:

- A daemon thread is a type of thread in Python that runs in the background and is automatically terminated when the main program finishes, without waiting for the thread to complete its execution. It is typically used for background tasks that do not need to prevent the program from exiting.

- Unlike non-daemon threads, daemon threads do not block the program from terminating. The Python interpreter exits as soon as the main program finishes, even if daemon threads are still running.

### Key Attributes of Daemon Threads:
#### daemon:
- A boolean attribute (True or False) indicating whether the thread is a daemon thread.
- Default: False (non-daemon thread).
- Set before starting the thread: Once the thread starts, the daemon attribute cannot be changed.
#### is_alive():
- Returns True if the thread is still alive (running).
- It can be used to check if a daemon thread is still running.
#### name:
- The name of the thread. By default, it is a system-generated name, but you can set a custom name when creating the thread.
#### ident:
- The unique identifier of the thread. It is assigned when the thread starts running and can be used to track the thread's execution.
#### native_id:
- The native thread ID assigned by the operating system, useful for OS-level debugging or inspection of the thread.

## Scenario: Teaching Sessions and Exam
### 1. Using Non-Daemon Thread

In [9]:
from threading import *
import time

def display():
    for i in range(10):
        print("Teaching session:", i)
        time.sleep(0.7)

# Create a non-daemon thread
t1 = Thread(target=display)
t1.start()

# Main thread handling the Exam
print("Exam Time!")
time.sleep(2)  # Simulate exam process
print("Exam is over")


Teaching session: 0
Exam Time!
Teaching session: 1
Teaching session: 2
Exam is over
Teaching session: 3
Teaching session: 4
Teaching session: 5
Teaching session: 6
Teaching session: 7
Teaching session: 8
Teaching session: 9


### Issues:
- The main thread finishes its work (exam), but the program doesn’t exit because the non-daemon thread (Teaching Session) continues running until it completes all iterations.
- This leads to unnecessary delay even though the main thread's work is done.

### 2. Using Daemon Thread:
- If this code doesn't work fine in jupyter notebook, copy it and run into another ide.

In [1]:
from threading import *
import time

def display():
    for i in range(10):
        print("Teaching session:", i)
        time.sleep(0.7)

# Create a daemon thread
t1 = Thread(target=display, daemon=True)
t1.start()

# Main thread handling the Exam
print("Exam Time!")
time.sleep(2)  # Simulate exam process
print("Exam is over")


Teaching session: 0
Exam Time!
Teaching session: 1
Teaching session: 2
Exam is over
Teaching session: 3
Teaching session: 4
Teaching session: 5
Teaching session: 6
Teaching session: 7
Teaching session: 8
Teaching session: 9


### Advantages with Daemon Thread:
- Non-Blocking: The main thread does not block, allowing it to finish its work even if the daemon thread is still running.
- Main Thread Finishes Early: The program prints "Main thread finished." immediately, even though the daemon thread continues running in the background (printing numbers).
- Daemon Thread Termination: If the main thread finishes, the program ends, even if the daemon thread hasn't finished printing all the numbers.

### Use Case for Daemon Threads:
- Daemon threads are useful for background tasks, such as logging, monitoring, or housekeeping tasks, where you don’t need to wait for the task to complete before finishing the program.
- For example, a logging thread running in the background can be a daemon thread that logs data without blocking the main application from exiting.

### Key points:
- It is possible to switch a thread between daemon and non-daemon states, but this must be done before starting the thread.
- The main thread, however, cannot be set as a daemon thread, as it is essential for program execution.